In [2]:
import tensorflow as tf
tf.__version__


'2.13.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [14]:
subject = 'Santa Catarina - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [15]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_SC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda,Santa Catarina - Consumo de Cimento (t)
0,2008,1.404443e+08,6.811481e+06,20.042997,1.249726e+08,8.020915,354.241699,0.802185,0.868622,5.483335e+08,...,22.705450,1015.709740,1159.562545,20.995392,16.887106,78.404177,2.205136,0.764435,0.777491,2653.587000
1,2009,1.414019e+08,6.842789e+06,20.044049,1.250753e+08,8.049936,431.347926,0.802866,0.870023,5.302764e+08,...,22.246273,1015.468602,1144.441684,20.980939,16.993726,78.934917,2.172407,0.766524,0.776296,2669.400000
2,2010,1.450816e+08,6.897548e+06,20.092849,1.259937e+08,8.013741,792.441715,0.803485,0.870869,5.046425e+08,...,21.990797,1015.572649,1157.010200,20.848998,16.800270,78.648476,2.054779,0.768517,0.775508,2852.759000
3,2011,1.483956e+08,6.888268e+06,20.118967,1.266496e+08,8.000656,1268.083588,0.804044,0.871172,4.854622e+08,...,21.342050,1015.696428,1175.275698,20.915915,16.747918,78.133882,1.801706,0.770407,0.775118,3087.810000
4,2012,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,4.760238,1549.260439,0.804247,0.870803,4.867661e+08,...,20.883838,1015.789692,1187.018145,21.084508,16.909840,78.138320,1.427605,0.771704,0.774917,3268.310000
5,2013,1.539868e+08,6.837970e+06,20.102531,1.275421e+08,5.187787,1645.417258,0.805268,0.870203,5.225848e+08,...,21.423516,1015.883075,1188.987224,21.299307,17.182298,78.463284,1.150329,0.774346,0.775826,3466.227000
6,2014,1.560820e+08,6.814799e+06,20.085277,1.277386e+08,5.511795,1598.981415,0.805624,0.868970,5.676413e+08,...,21.963483,1015.872526,1152.319790,21.397543,17.331617,78.730613,1.252238,0.775791,0.776479,3246.749286
7,2015,1.574146e+08,6.792567e+06,20.038056,1.276578e+08,5.792915,1488.521029,0.805509,0.867217,5.179392e+08,...,21.813817,1015.950998,1142.708354,21.308544,17.138396,78.237612,1.540937,0.776440,0.777102,3017.084229
8,2016,1.585019e+08,6.760156e+06,20.026681,1.277375e+08,5.798844,1430.320858,0.807604,0.875108,4.524144e+08,...,22.023495,1016.049222,1167.145418,21.317634,16.960969,77.364404,1.763965,0.782298,0.769848,2776.217486
9,2017,1.591962e+08,6.740058e+06,20.047554,1.279508e+08,5.526597,1433.051517,0.807023,0.872664,4.213689e+08,...,21.801864,1016.030235,1163.903842,21.473728,17.031652,76.967201,1.872343,0.782053,0.770470,2891.489000


In [16]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
0,1.404443e+08,6.811481e+06,20.042997,1.249726e+08,8.020915,354.241699,0.802185,0.868622,5.483335e+08,2.546644e+09,...,11.760983,22.705450,1015.709740,1159.562545,20.995392,16.887106,78.404177,2.205136,0.764435,0.777491
1,1.414019e+08,6.842789e+06,20.044049,1.250753e+08,8.049936,431.347926,0.802866,0.870023,5.302764e+08,2.679945e+09,...,9.472585,22.246273,1015.468602,1144.441684,20.980939,16.993726,78.934917,2.172407,0.766524,0.776296
2,1.450816e+08,6.897548e+06,20.092849,1.259937e+08,8.013741,792.441715,0.803485,0.870869,5.046425e+08,2.891930e+09,...,9.332485,21.990797,1015.572649,1157.010200,20.848998,16.800270,78.648476,2.054779,0.768517,0.775508
3,1.483956e+08,6.888268e+06,20.118967,1.266496e+08,8.000656,1268.083588,0.804044,0.871172,4.854622e+08,3.091988e+09,...,10.985689,21.342050,1015.696428,1175.275698,20.915915,16.747918,78.133882,1.801706,0.770407,0.775118
4,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,4.760238,1549.260439,0.804247,0.870803,4.867661e+08,3.189506e+09,...,8.159014,20.883838,1015.789692,1187.018145,21.084508,16.909840,78.138320,1.427605,0.771704,0.774917
5,1.539868e+08,6.837970e+06,20.102531,1.275421e+08,5.187787,1645.417258,0.805268,0.870203,5.225848e+08,3.093872e+09,...,7.913561,21.423516,1015.883075,1188.987224,21.299307,17.182298,78.463284,1.150329,0.774346,0.775826
6,1.560820e+08,6.814799e+06,20.085277,1.277386e+08,5.511795,1598.981415,0.805624,0.868970,5.676413e+08,2.779340e+09,...,10.350442,21.963483,1015.872526,1152.319790,21.397543,17.331617,78.730613,1.252238,0.775791,0.776479
7,1.574146e+08,6.792567e+06,20.038056,1.276578e+08,5.792915,1488.521029,0.805509,0.867217,5.179392e+08,2.561731e+09,...,12.454398,21.813817,1015.950998,1142.708354,21.308544,17.138396,78.237612,1.540937,0.776440,0.777102
8,1.585019e+08,6.760156e+06,20.026681,1.277375e+08,5.798844,1430.320858,0.807604,0.875108,4.524144e+08,2.516859e+09,...,13.111908,22.023495,1016.049222,1167.145418,21.317634,16.960969,77.364404,1.763965,0.782298,0.769848
9,1.591962e+08,6.740058e+06,20.047554,1.279508e+08,5.526597,1433.051517,0.807023,0.872664,4.213689e+08,2.497710e+09,...,9.479812,21.801864,1016.030235,1163.903842,21.473728,17.031652,76.967201,1.872343,0.782053,0.770470


In [17]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2669.400000
1     2852.759000
2     3087.810000
3     3268.310000
4     3466.227000
5     3246.749286
6     3017.084229
7     2776.217486
8     2891.489000
9     2976.485000
10    3258.027000
11    3537.032000
12    4089.333000
13    4255.808000
14            NaN
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [18]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
0,-1.788831,-0.053289,-0.765812,-1.946661,1.383658,-2.039420,-1.752749,-0.621103,1.204752,-0.990234,...,0.984445,1.435761,-0.708446,-0.446366,-0.967915,-0.373775,0.579127,1.332149,-1.644452,1.173188
1,-1.646154,0.543626,-0.732922,-1.849054,1.406794,-1.864637,-1.314663,-0.021470,0.820657,-0.546688,...,-0.051710,0.887657,-1.954083,-1.451705,-1.034435,0.196789,1.104304,1.233818,-1.312944,0.756047
2,-1.097922,1.587678,0.792736,-0.976209,1.377939,-1.046116,-0.916498,0.340864,0.275392,0.158672,...,-0.115145,0.582703,-1.416611,-0.616063,-1.641666,-0.838463,0.820866,0.880425,-0.996740,0.480739
3,-0.604178,1.410733,1.609275,-0.352824,1.367508,0.032060,-0.556764,0.470555,-0.132596,0.824346,...,0.633403,-0.191684,-0.777213,0.598353,-1.333696,-1.118619,0.311667,0.120113,-0.696766,0.344683
4,-0.161412,1.129186,1.550236,0.121740,-1.215809,0.669427,-0.426008,0.312334,-0.104860,1.148829,...,-0.646476,-0.738636,-0.295440,1.379072,-0.557774,-0.252116,0.316058,-1.003809,-0.491049,0.274475
5,0.228858,0.451761,1.095422,0.495478,-0.874960,0.887393,0.230528,0.055613,0.657046,0.830615,...,-0.757614,-0.094441,0.186945,1.509990,0.430797,1.205911,0.637615,-1.836837,-0.071805,0.592030
6,0.541016,0.009964,0.555991,0.682235,-0.616655,0.782133,0.459321,-0.472188,1.615453,-0.215960,...,0.345772,0.550099,0.132453,-0.927914,0.882911,2.004971,0.902142,-1.530670,0.157572,0.819692
7,0.739557,-0.413904,-0.920302,0.605403,-0.392541,0.531744,0.385546,-1.222320,0.558230,-0.940033,...,1.298413,0.371448,0.537815,-1.566949,0.473308,0.970972,0.414308,-0.663324,0.260600,1.037237
8,0.901552,-1.031862,-1.275923,0.681176,-0.387815,0.399817,1.733415,2.155314,-0.835563,-1.089341,...,1.596124,0.621735,1.045202,0.057796,0.515145,0.021497,-0.449746,0.006727,1.190164,-1.494981
9,1.004993,-1.415041,-0.623358,0.883876,-0.604855,0.406006,1.359405,1.108906,-1.495939,-1.153056,...,-0.048438,0.357181,0.947123,-0.157727,1.233540,0.399746,-0.842785,0.332330,1.151189,-1.277683


In [19]:
reshaped_train_input = dfToInputRNN(train_input)

In [20]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.85831341, -0.98039984, -0.61699777,  0.76318959,
         -0.69841501,  0.67107459,  0.11397466, -1.1290481 ,
         -1.0453709 ,  1.21859064, -0.99976018,  0.79428371,
          1.15575749, -0.52559858,  1.30260559, -0.55253046,
         -0.25130359,  1.38787176,  1.47154451, -1.80649616,
          1.09296291,  0.94281138,  0.52836084,  0.61258103,
          0.87656257,  1.47801536,  1.71982086,  1.28221181,
          1.39086472,  1.58209844,  1.08872091,  0.55212962,
         -1.81398118,  1.44113674,  1.11191094,  1.30470835,
          0.86390516, -1.67365968, -1.70738939, -1.55813196,
         -0.48498301, -0.75714639,  0.40770893, -0.27394734,
          2.02323219,  1.2908951 ,  1.70465513,  1.81148084,
          1.39968653, -1.36191442,  1.28024593,  1.11054795,
          0.12662799,  1.40414112,  1.40468999,  1.45643959,
          1.58509336,  1.32283157,  1.30222047,  1.55138602,
          1.91801602, -0.77489784,  2.32098884, -1.1942541 ,
         -1.17081042, -1

In [26]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     2669.400000
1     2852.759000
2     3087.810000
3     3268.310000
4     3466.227000
5     3246.749286
6     3017.084229
7     2776.217486
8     2891.489000
9     2976.485000
10    3258.027000
11    3537.032000
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [27]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    3537.032
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [28]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
12,0.569016,-0.799091,-1.589659,0.368609,-0.769661,1.066131,-1.10113,-2.524529,1.108405,2.635823,...,-2.260447,-1.929808,0.722827,0.501119,-0.180244,-1.944793,-1.604836,0.135558,0.9946,-1.378467


In [29]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.56901595, -0.79909108, -1.58965928,  0.36860911,
         -0.76966101,  1.06613121, -1.10113046, -2.52452908,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738, -1

In [30]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    4089.333
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [97]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=1000, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(592, activation='tanh', return_sequences=True, input_shape=(train_input.shape[1], 
                                                                                         train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(74, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=5000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [98]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, 0, seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [ ]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2682371651, 2801996629, 2679807246, 1006690134, 2677345024, 3635268601, 2580000239, 2185832736, 1911829959, 3861660978, 3529724710, 4250266591, 2526948068, 4030154065, 2202392022, 208646911, 3394006981, 2624446185, 2580372955, 2427258994, 3791073206, 3100681134, 110518587, 2953959659, 1241249905, 3840647138, 324507367, 2598414545, 1069621277, 4095212868, 2476077407, 2890147516, 1013560778, 1166736957, 3766876653, 4141232848, 3623444884, 3911496658, 3901976228, 164688977, 673480246, 4266898811, 1663623082, 201962554, 2120799338, 380338767, 450731048, 3702342143, 778544411, 1752573259, 949157726, 3650532913, 1015883006, 1148093029, 2691596350, 3757305270, 3393434480, 4128773961, 222113261, 1789118307, 977559626, 3305636156, 1022885889, 4159816224, 2607597647, 891380094, 183722546, 77063111, 361056738, 769662956, 2800617907, 1322132070, 4125833067, 3511901520, 697918033, 4223539154, 1388418584, 3486410391, 574823551, 337333580, 1689855610, 1470406939, 3266158146, 4256200565, 1337069364, 

In [ ]:
trained_model, history = lstm_model(reshaped_train_input, 
                                    train_target, want_verbose=1, seed=winner_seed)

In [ ]:
prediction = trained_model.predict(reshaped_test_input)[0][0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

In [ ]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

In [ ]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = lstm_model(dfToInputRNN(train_input), 
                                       train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0][0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [ ]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

In [ ]:
display(targets)
display(predictions)

In [ ]:
mae = mean_absolute_error(predictions, targets)
mae

In [ ]:
porcentage = mae/np.mean(targets)
porcentage